In [ ]:
import math
#import random
#import re
from functools import reduce
from itertools import permutations
#import sympy
#sympy.init_printing()

import sys; sys.path.append("../modules")
from Permutation import *

# import pandas as pd
from tqdm.notebook import tqdm
# ----- Debugger -----
# from IPython.core.debugger import Pdb; Pdb().set_trace()

from knot_floer_homology import *
#help(pd_to_hfk)

## Creating PD-codes for knots with given crossing number

In [ ]:
for a in range(4):
    print(f"{a} --> {bin(a & 0b01), bin(a & 0b10), bool(a & 0b01), bool(a & 0b10)}")
    print('ON' if a & 0b01 else 'OFF', 'ON' if a & 0b10 else 'OFF')
    print(a not in range(4))

In [ ]:
b = 2**2+2**1+2**0; print(bin(b))

for i in range(3):
    print(bin(b & 2**i))

In [ ]:
for sign_bits in range(4**3):
    print(f"{sign_bits:#08b} --> {[((sign_bits >> 2*i) & 0b11) for i in range(3)]}")

In [ ]:
N = 6
pd_list = []

def cross(x: int, y: int, sign: int):
    if sign not in range(4): raise(ArgumentError) # signs are 0b00, 0b01, 0b10, 0b11
    if (sign & 0b10):
        ft, lt = (x, x%(2*N)+1), (y, y%(2*N)+1)
    else:
        ft, lt = (y, y%(2*N)+1), (x, x%(2*N)+1)
    if (sign & 0b01):
        pd = (ft[0], lt[0], ft[1], lt[1])
    else:
        pd = (ft[0], lt[1], ft[1], lt[0])
    return pd    

for perm in tqdm(permutations(range(N+1,2*N+1), N), total=math.factorial(N)): # iteratation on permutations on {N+1, ... ,2*N+1} 
    if (perm[0] != 2*N) and (perm[N-1] != N+1):
        for signs in range(2**(2*N-1)):
            pd = [cross(i+1, perm[i], ((signs >> 2*i) & 0b11)) for i in range(N)]
#            print(pd)
            cs = [Cycle([x-1 for x in tup], 2*N) for tup in pd]
            pd_perm = reduce((lambda x,y: x*y), cs)
            num_of_orbits = len(cycle_decomp(pd_perm))
            if num_of_orbits - N == 2:
#                pd_list.append(pd)
                hfk = pd_to_hfk(pd)
                if hfk['fibered'] and hfk['seifert_genus'] > 0:
                    print(f"{pd=}\n hfk: {hfk['ranks']}, {hfk['seifert_genus']}\n")
                    
#print("\n")
#for pd in pd_list:
#    print(f"{pd}")
#print(len(pd_list))

## scratch

In [ ]:
hfk = pd_to_hfk([[1,5,2,4],[5,3,6,2],[3,1,4,6]])
print(f"{hfk}")

In [ ]:
hfk = pd_to_hfk([[1,4,2,5],[3,1,4,6],[5,3,6,2]])
print(f"{hfk}")

In [ ]:
hfk = pd_to_hfk([[1,5,2,4],[5,2,6,3],[3,1,4,6]])
print(f"{hfk}")

In [ ]:
[bin(x) for x in range(2**3)]

In [ ]:
k= 6
for i in range(3):
    flag = bin(k & 2**i)
    print(flag)

In [ ]:
print(cs[0], cs[0].image)

print(cs[1]*cs[0], (cs[1]*cs[0]).image)